In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys 
sys.path.append('../')

In [3]:
import torch
import torch.nn as nn
from src.transactions_qa.utils import get_projections_maps, get_exponent_number, get_mantissa_number

In [4]:
inp = torch.arange(10)
buck = torch.arange(0, 10, step=2)

In [5]:
# !pip uninstall rtdl -y

In [6]:
indices = torch.bucketize(inp, buck, right=False)
# masked_indices = 

num_buckets = len(buck) + 1

new_indices = torch.clamp(indices -1, 0, num_buckets-2)

matrix = torch.tril(torch.ones(num_buckets, num_buckets))

In [7]:
original_matrix = matrix[indices]

In [8]:
bucket_sizes = buck.diff()
buck_indices = torch.clamp(new_indices, 0, num_buckets - 3)
size_down = bucket_sizes[buck_indices]

adding = (inp - buck[new_indices]) / size_down
mask_borders = (indices != 0) & (indices != num_buckets - 1)
adding[~mask_borders] = 1.0

In [9]:
mask_matrix = original_matrix.sum(1).long() - 1

In [10]:
mask_matrix.shape

torch.Size([10])

In [11]:
original_matrix[torch.arange(10), mask_matrix] *= adding

In [12]:
original_matrix.shape

torch.Size([10, 6])

In [13]:
weight = nn.Linear(6, 385)

In [14]:
basic_embedding =  original_matrix @ weight.weight

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x6 and 385x6)

In [ ]:
import torch.nn.functional as F

In [ ]:
weight(original_matrix)

tensor([[-0.3753,  0.3176, -0.3136,  ..., -0.1803,  0.3739, -0.0013],
        [-0.2016,  0.2654, -0.3112,  ..., -0.0968,  0.2023,  0.0351],
        [-0.0278,  0.2132, -0.3087,  ..., -0.0134,  0.0306,  0.0715],
        ...,
        [ 0.7058, -0.3745, -0.1778,  ...,  0.1686, -0.3010,  0.3373],
        [ 0.6927, -0.4647, -0.0395,  ...,  0.1275, -0.3487,  0.1577],
        [ 0.8186, -0.7694,  0.3162,  ...,  0.2795,  0.0012,  0.5489]],
       grad_fn=<AddmmBackward0>)

In [15]:
basic_embedding.shape

NameError: name 'basic_embedding' is not defined

In [16]:
class PiecewiseLinearEmbedding(nn.Module):
    def __init__(self, embedding_dim, buckets):
        super().__init__()
        self.buckets = buckets 
        self.num_buckets = len(buckets) + 1

        self.layer = nn.Linear(self.num_buckets, embedding_dim)
        self.matrix = torch.tril(torch.ones(self.num_buckets, self.num_buckets))

        self.bucket_sizes = self.buckets.diff()
        
    def forward(self, x):
        batch_size = x.shape[0]
        device = x.device

        indices = torch.bucketize(x, self.buckets, right=False)
        new_indices = torch.clamp(indices - 1, 0, self.num_buckets - 2)
        original_matrix = self.matrix[indices]

        buck_indices = torch.clamp(new_indices, 0, self.num_buckets - 3)
        size_down = self.bucket_sizes[buck_indices]

        adding = ((x - self.buckets[new_indices]) / size_down)

        mask_borders = ((indices != 0) & (indices != self.num_buckets - 1))
        adding[~mask_borders] = 1.0
        mask_matrix = original_matrix.sum(1).long() - 1

        original_matrix[torch.arange(batch_size), mask_matrix] *= adding

        return self.layer(original_matrix)

In [55]:
batch = torch.linspace(-3.5, 3.5, steps=200)
buckets = torch.linspace(-3, 3, steps=10)

In [56]:
model = PiecewiseLinearEmbedding(128, buckets)

In [57]:
class NumEmbedding(nn.Module):
    def __init__(self, embedding_dim, buckets) -> None:
        super().__init__()

        self.mantissa_embedding = PiecewiseLinearEmbedding(embedding_dim, buckets)
        self.exponent_embedding = nn.Embedding(17, embedding_dim)

    def forward(self, x):
        exponent = get_exponent_number(x) + 8
        mantissa = get_mantissa_number(x)

        embedding_exponent = self.exponent_embedding(exponent)
        embedding_mantissa = self.mantissa_embedding(mantissa)

        return torch.cat([embedding_mantissa, embedding_exponent])

In [58]:
numemb = NumEmbedding(385, buckets)

In [60]:
numemb(batch).shape

torch.Size([400, 385])

In [52]:
import re

questions = ["Will the amount of the next transaction will be more than 0.54", "Will the amount of the next transaction will be more than 0.54"]

num_values = [re.findall("\d+\.\d+", question) for question in questions]
numerics = [re.sub("\d+\.\d+", '<extra_id_0>', string) for string in questions]

In [53]:
numerics

['Will the amount of the next transaction will be more than <extra_id_0>',
 'Will the amount of the next transaction will be more than <extra_id_0>']

In [48]:
values = []
for value_list in num_values:
    for value in value_list:
        values.append(float(value))

In [51]:
torch.tensor(values)

tensor([0.5400, 0.5400])

NameError: name 'tqa_model' is not defined

In [17]:
###

SyntaxError: unexpected EOF while parsing (1035302448.py, line 1)

### New metrics

In [18]:
from torchmetrics.text import BLEUScore, ROUGEScore

In [49]:
preds  = torch.round(torch.randn(10), decimals=2)
targets =  torch.round(torch.randn(10), decimals=2)

def transform_for_text_metrics(preds, targets):
    new_preds = list(map(lambda x: str(round(x.item(), 2)).replace("", " ")[1: -1], preds))
    new_targets = list(map(lambda x: str(round(x.item(), 2)).replace("", " ")[1: -1], targets))
    return  new_preds, new_targets

In [35]:
from torchmetrics.text.rouge import ROUGEScore
preds = ["0 1 2 2 3"]
target = ["0 2 2 2 3"]
rouge = ROUGEScore()
from pprint import pprint
pprint(rouge(preds, target))


{'rouge1_fmeasure': tensor(0.8000),
 'rouge1_precision': tensor(0.8000),
 'rouge1_recall': tensor(0.8000),
 'rouge2_fmeasure': tensor(0.5000),
 'rouge2_precision': tensor(0.5000),
 'rouge2_recall': tensor(0.5000),
 'rougeL_fmeasure': tensor(0.8000),
 'rougeL_precision': tensor(0.8000),
 'rougeL_recall': tensor(0.8000),
 'rougeLsum_fmeasure': tensor(0.8000),
 'rougeLsum_precision': tensor(0.8000),
 'rougeLsum_recall': tensor(0.8000)}


### Words to numbers

In [50]:
import re



def text2float(textnum, decimal='dot'):
    numwords = {}

    units = [ "zero", "one", "two", "three", "four", "five", "six",
            "seven", "eight", "nine"]
            
    bigger_units =["ten", "eleven", "twelve",
            "thirteen", "fourteen", "fifteen", "sixteen", "seventeen",
            "eighteen", "nineteen"]

    tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", 
            "seventy", "eighty", "ninety"]

    scales = ["hundred", "thousand", "million", "billion", "trillion", 
            'quadrillion', 'quintillion', 'sexillion', 'septillion', 
            'octillion', 'nonillion', 'decillion' ]

    numwords["and"] = (1, 0)
    for idx, word in enumerate(units+bigger_units): numwords[word] = (1, idx)
    for idx, word in enumerate(tens): numwords[word] = (1, idx * 10)
    for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 
            'eighth':8, 'ninth':9, 'twelfth':12}
    ordinal_endings = [('ieth', 'y'), ('th', '')]
    current = result = 0
    

    if "dot"  in textnum:
        first, second = re.split(decimal, textnum)
        first_tokens = re.split(r"[\s-]+", first.rstrip())
        second_tokens = re.split(r"[\s-]+", second.lstrip())
    else:
        first_tokens = re.split(r"[\s-]+", textnum)
        
    for word in first_tokens:
        if word in ordinal_words:
            scale, increment = (1, ordinal_words[word])
        else:
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)

            if word not in numwords:
                raise Exception("Illegal word: " + word)

            scale, increment = numwords[word]

        if scale > 1:
            current = max(1, current)

        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0

    result += current

    if "dot" in textnum:
        multi = 0.1
        for word in second_tokens:
            if word not in units:
                raise Exception("Illegal word in fractional: " + word)
            scale, increment = numwords[word]
            result += multi * increment
            multi *= 0.1

    return result 